# DSCI 522


## Lecture 8 - Environment management: containerization with Docker part III & Reproducibility wrap-up

### 2018-02-06

## A note from the last couple lectures:

#### Running rocker Docker images from the Shell instead of RStudio

The default behaviour for the rocker Docker images, including the `rocker/tidyverse` image is to launch RStudio. You can override this as follows:

```
docker run -it -e PASSWORD=yourpassword --rm rocker/tidyverse /bin/bash

```

The key is to add `/bin/bash` after the you call the image. This should also work for other images that have other default start-up behaviours.

Why does this image not enter a bash terminal as the others we have been working with do? This is because this Dockerfile (well it's base image's Dockerfile) has `CMD ["\init"]` that sets the containers default behaviour to launching the RStudio server GUI when it is run. We can override this by specifying `/bin/bash` after we speficy the Docker image in the `docker run` command.

#### Example of Milestone 4 available

- https://github.com/ttimbers/breast_cancer_predictor/tree/v4.0

# Learning Objectives

## By the end of the lecture, students should be able to:

1. [Build a Docker image from a Dockerfile on Docker Hub](#Build-a-Docker-image-from-a-Dockerfile-on-Docker-Hub)
1. [Run a Docker container non-interactively](#Running-a-Docker-container-non-interactively)
- [Version Docker images and report software and package versions](#Version-Docker-images-and-report-software-and-package-versions)
- [Use conda to share data analysis software depenedencies (R, Python, packages, etc)](#conda----a-tool-to-create-isolated-computing-environments.)
- [Articulate why reproducibility is important for data analysis](#Class-discussion)
- [Choose the appropriate level or reproducibility workflows/tooling based on project complexity and desired shareability](#Class-discussion)

## Build a Docker image from a Dockerfile on Docker Hub

Building a Docker image from a Dockerfile on Docker Hub lets others trust your image. We can do this by pushing our Dockerfile to a repository on GitHub.com and then linking a Docker Hub repository to that.

#### 1. Push your Dockerfile to a repository on GitHub.com

We recommend putting the Dockerfile in the root of your project:

<img src="img/github-dockerfile.png" width=600>

#### 2. Set-up a repository on Docker Hub for the Docker image you want to build

Click on "Create Repository":

<img src="img/docker-ab-1.png" width=600>

Choose your Docker Hub username and give the Docker Hub repository a name (this will become the image name). Then click on "Connected":

<img src="img/docker-ab-5.png" width=600>

*note: you will have to have already linked your GitHub.com account to your Docker Hub account (you only need to do this once). Documentation on how to do this is here: https://success.docker.com/article/how-do-i-link-my-github-account*

Select your GitHub.com username and the GitHub.com repository that contains the Dockerfile you want to build. Then click the "+" beside "BUILD RULES":

<img src="img/docker-ab-6.png" width=600>



Here enter any necessary information about the build (e.g., give it a version tag, and/or change the path to point to the Dockerfile if it doesn't live in the GitHub repository root), and then click "Create & Build":

<img src="img/docker-ab-7.png" width=600>

The orange button indicates that the process of building the image is pending. A blue button indicates the build is in process. 

<img src="img/docker-ab-8.png" width=600>

And finally, a turquise button with a checkmark indicates that the image was successfully built!

<img src="img/docker-ab-9.png" width=300>

## Running a Docker container non-interactively

So far we have been running our containers interactively, but sometimes we want to automate further and run things non interactively. We do this be dropping the `-it` flag from our `docker run` command as well as calling a command or a script after the docker image is specified.

The general form for for running things non-interactively is this:

```
docker run --rm -v PATH_ON_YOUR_COMPUTER:VOLUME_ON_CONTAINER DOCKER_IMAGE PROGRAM_TO_RUN PROGRAM_ARGUMENTS
```

For example, let's use the container we built last class to non-interactively run a `cowsay::say` function call to print some asci art with a cute message! 

```
$ docker run --rm ttimbers/dockerfile-practice:v1.0 Rscript -e "library(cowsay); say('Snow again this week?', 'snowman')"
```

And if succesfful, we should get:

```
----- 
Snow again this week? 
 ------ 
    \   
     \
     _[_]_
      (")
  >--( : )--<
    (__:__) [nosig]
```  

*note - I will use the version of the image that exists on Docker Hub, but you are free to use the one we built last class, they should be the same.*

Now that was a silly example, but this can be made powerful so that we can run an analysis pipeline, such as a Makefile non-interactively using Docker! Here's a demo we can try: https://github.com/ttimbers/data_analysis_pipeline_eg/tree/v4.0

### Exercise 1: 

Download https://github.com/ttimbers/data_analysis_pipeline_eg/archive/v4.0.zip, unzip it and navigate to the root of the project directory, try to run the analysis via `make all`.

### Exercise 2: 

Now try to run the analysis using Docker via:

```
docker run --rm -v /$(pwd):/home/rstudio/data_analysis_eg ttimbers/data_analysis_pipeline_eg make -C /home/rstudio/data_analysis_eg all
```

*note - windows users must use Git Bash, set Docker to use Linux containers, and have shared their drives with Docker (see docs [here](https://token2shell.com/howto/docker/sharing-windows-folders-with-containers/)) for this to work*


If that was too easy (i.e., both worked), try running this analysis with and without Docker: https://github.com/ttimbers/breast_cancer_predictor

Now let's try a more complex project (the breast cancer predictor):

### Exercise 3: 

Download https://github.com/ttimbers/breast_cancer_predictor/archive/v4.0.zip, unzip it and navigate to the root of the project directory, try to run the analysis via `make clean` (to reset the project state) and `make all` (to run the analysis).

### Exercise 4: 

Now try to run reset the project state using Docker via:

```
docker run --rm -v /$(pwd):/home/rstudio/breast_cancer_predictor ttimbers/bc_predictor:v4.0 make -C /home/rstudio/breast_cancer_predictor clean
```

Now try to run the analysis using Docker via:

```
docker run --rm -v /$(pwd):/home/rstudio/breast_cancer_predictor ttimbers/bc_predictor:v4.0 make -C /home/rstudio/breast_cancer_predictor all
```

*note - windows users must use Git Bash, set Docker to use Linux containers, and have shared their drives with Docker (see docs [here](https://token2shell.com/howto/docker/sharing-windows-folders-with-containers/)) for this to work*

What we just did was run our Makefile non-interactively. We had to change a few things in our Make command to make this work (no pun intended). 

First, we run our Makefile from a different directory. So to point at a Makefile using a path we use `make -C PATH_TO_MAKEFILE`. Following that we added our Makefile target, in this case `all`. 

If we wanted to reset our analysis to a clean state where no analysis had been done, we could just change the target from `all` to `clean` and everything else would stay the same:

```
docker run --rm -v /$(pwd):/home/rstudio/data_analysis_eg ttimbers/data_analysis_pipeline_eg make -C /home/rstudio/data_analysis_eg make
```



## Version Docker images and report software and package versions

It is easier to create a Docker image from a Dockerfile and tag it (or use it's digest) than to control the version of each thing that goes into your Docker image.

- tags are human readable, however they can be associated with different builds of the image (potentially using different Dockerfiles...)
- digests are not human readable, but specify a specific build of an image

Example of how to pull using a tag: 
```
docker pull ttimbers/dockerfile-practice:v1.0
```

Example of how to pull using a digest:
```
docker pull ttimbers/dockerfile-practice@sha256:cc512c9599054f24f4020e2c7e3337b9e71fd6251dfde5bcd716dc9b1f8c3a73
```

Tags are specified when you build on Docker Hub on the Builds tab under the Configure automated builds options. Digests are assigned to a build. You can see the digests on the Tags tab, by clicking on the "Digest" link for a specific tag of the image.

### How to get the versions of your software in your container

Easiest is to enter the container interactively and poke around using the following commands:

- `python --version` and `R --version` to find out the versions of Python and R, respectively
- `pip freeze` in the bash shell to find out Python package versions
- Enter R and load the libraries used in your scripts, then use `sessionInfo()` to print the package versions

## But I want to control the versions!

### How to in R:

#### The Rocker team's strategy

This is not an easy thing, but the Rocker team has made a concerted effort to do this. Below is their strategy:

> Using the R version tag will naturally lock the R version, and also lock the install date of any R packages on the image. For example, rocker/tidyverse:3.3.1 Docker image will always rebuild with R 3.3.1 and R packages installed from the 2016-10-31 MRAN snapshot, corresponding to the last day that version of R was the most recent release. Meanwhile rocker/tidyverse:latest will always have both the latest R version and latest versions of the R packages, built nightly.

See [VERSIONS.md](https://github.com/rocker-org/rocker-versioned/blob/master/VERSIONS.md) for details, but in short they use the line below to lock the R version (or view in r-ver Dockerfile [here](https://github.com/rocker-org/rocker-versioned/blob/c4a9f540d4c66a6277f281be6dcfe55d3cb40ec0/r-ver/3.6.1.Dockerfile#L76) for more context):
```  
    && curl -O https://cran.r-project.org/src/base/R-3/R-${R_VERSION}.tar.gz \
```

And this line to specify the CRAN snapshot from which to grab the R packages (or view in r-ver Dockerfile [here](https://github.com/rocker-org/rocker-versioned/blob/c4a9f540d4c66a6277f281be6dcfe55d3cb40ec0/r-ver/3.6.1.Dockerfile#L121) for more context):
```
    && Rscript -e "install.packages(c('littler', 'docopt'), repo = '$MRAN')" \
```

What is `MRAN`? Everyday Microsoft takes daily snapshots of the CRAN R package repository and archives them! We can choose a day in time and install all our packages from there, ensuring that everytime we build our Docker image we end up with the same version of the R package, no matter which day we build it on.

So, a simpler example from above, without using an variable (like the Rocker group does is):

```
RUN Rscript -e "install.packages('gapminder', repos = 'https://mran.revolutionanalytics.com/snapshot/2016-10-19')"
```

#### A new thing that might be useful in the future?

[renv](https://rstudio.github.io/renv/articles/renv.html) - this is new! 🎉

### How to in Python:

Python version:

- `conda`  to specify an install of specific Python version, either when downloading (see example [here](https://github.com/ContinuumIO/docker-images/blob/8e10242c6d7804a0e991a9d9d758e25b340f4fce/miniconda3/debian/Dockerfile#L10), or after downloading with `conda install python=3.6`).
- Or you can install a specific version of Python yourself, as they do in the Python official images (see [here](https://github.com/docker-library/python/blob/master/3.7/stretch/slim/Dockerfile) for example), but this is more complicated.

For Python packages, there are a few tools:
- conda (via `conda install scipy=0.15.0` for example)
- pip (via `pip install scipy=0.15.0` for example)

## Take home messages:

- At a minimum, tag your Docker images or reference image digests
- If you want to version installs inside the container, use base images that version R & Python, and add what you need on top in a versioned manner!

# Overview of tools for managing mutliple environments and package dependencies

### Language agnostic (polyglot):

- Docker
- `conda` 

### Language specific:
- R
    - ~~`packrat`~~
    - [renv](https://rstudio.github.io/renv/articles/renv.html) - this is new! 🎉
    
- Python
    - `python-poetry`
    - `pipenv`
    - `virtualenv`
    
There are many to choose from!

## `conda` -  a tool to create isolated computing environments.

In addition to installing packages, `conda` is a tool to create isolated computing environments, which makes it easy to manage, record and ship dependencies. You have probably used conda most in the context of Python, but since you can install R and R packages with conda, you can use it for this (and other) languages too.


With the virtual environment created by `conda` you can specify the version of Python/R you want to use, as well as the version of the Python/R packages.

You can easily share this virtual environment with others who might want to use your code (or yourself if you need to use the code on another machine) by creating and sharing an `environment.yml` file.

Similar to Docker, `conda` packages installed from inside the isolated environment are only available to that isolated environment, and packages installed outside of the isolated environment are not available to the isolated environment. 

### Setting up a virtual environment for a Python project with conda

To create a new virtual environment, you use the `conda` command along with the `--name` argument (to name the virtual environment) and you can specify the version of Python (2 or 3):

```
conda create --name <environment_name> python=2
```

When you do this, `conda` automatically installs the packages and/or programs listed after the `<environment_name>`, and you will be prompted to choose whether to continue or not.

```
The following NEW packages will be INSTALLED:

    ca-certificates: 2018.03.07-0           
    certifi:         2018.10.15-py27_0      
    libcxx:          4.0.1-hcfea43d_1       
    libcxxabi:       4.0.1-hcfea43d_1       
    libedit:         3.1.20170329-hb402a30_2
    libffi:          3.2.1-h475c297_4       
    ncurses:         6.1-h0a44026_1         
    openssl:         1.1.1a-h1de35cc_0      
    pip:             18.1-py27_0            
    python:          2.7.15-h8f8e585_4      
    readline:        7.0-h1de35cc_5         
    setuptools:      40.6.2-py27_0          
    sqlite:          3.25.3-ha441bb4_0      
    tk:              8.6.8-ha441bb4_0       
    wheel:           0.32.3-py27_0          
    zlib:            1.2.11-h1de35cc_3      

Proceed ([y]/n)?
```

### Activate the virtual environment to use it

To use the virtual environment, you need to activate it. Otherwise you computer will just use the default Python. To activate the environment, use the following commands:

**Linux, OS X:** `source activate <environment_name>`

**Windows:** `activate <environment_name>`

*note for those using Windows - conda for package management doesn't play nice in GitBash... so I would suggest using CMD in the short term and Docker in the long term to make this work more smoothly...*

### Deactivate the virtual environment when you are done

Once you are done working on the project, you need to deactivate the environment so you can again use the default Python, to do so, use the following commands:

**Linux, OS X:** `conda deactivate`

**Windows:** `conda deactivate`

### Specifying more than one thing

You can also pass `conda` a list of other packages to install when you create the environment. For example, to create an environment called `data_science` with the `numpy`, `pandas` and `altair` package installed you can do the following:

```
conda create --name data_science python=3 numpy pandas altair

```

### Managing R & Python & Make possible!

```
conda create --name polyglot python=3 R make
```

### Installing packages within a `conda` virtual environment

You can add install new/additional packages to a `conda` virtual environment, the same way you install any package in Python at the command line, using either:

```
pip install <package_name>
```
or
```
conda install <package_name>
```

*Reminder - the key difference is that these packages will only be available to that specific Python project where you installed the package.*

### Shipping your dependencies with a conda virtual environment

You can create a file called `environment.yml` and add it to the root of your project that conda can use to create a virtual environment. 

For example, if our project dependend on Python 2.7 and a package called `screed` then our  `environment.yml` file would look like:

```
name: bio4
dependencies:
- python=2.7
- pip:
  - screed==0.9
```

We can then use the commands below to create that environment from that file:

```
conda env create -f environment.yml
```

### Shipping your dependencies with a conda virtual environment

To get a record of all the packages you have in the isolated environment, you can use `conda env export`. It is advisable to re-direct the output of this command to a file called `environment.yml`.

```
conda env export > environment.yml
```

*note - this needs to be done from inside the activated virtual environment.*

#### What to include in your `environment.yml` file?

Although you can automatically generate these, I still strongly suggest you curate them afterwards. This is because conda, by default includes OS-dependent packages and OS-dependent, conda-specific versioning of the packages. If you want someone on another OS to be able to use your `environment.yml` file YOU need to remove these.

For example, conda generates this:
```
name: bio2
channels: !!python/tuple
- defaults
dependencies:
- pip=8.1.2=py27_0
- python=2.7.12=0
- setuptools=27.2.0=py27_1
- vs2008_runtime=9.00.30729.1=2
- wheel=0.29.0=py27_0
- pip:
  - bz2file==0.98
  - screed==0.9
prefix: C:\Program Files\Anaconda3\envs\foo2
```

And I would curate it to this:
```
name: bio2
dependencies:
- python=2.7.12
- pip:
  - screed==0.9
```

### Dealing with R dependencies that are not on conda yet (but are on CRAN)

**WARNING - THERE BE DRAGONS HERE**, and so I cannot wholeheartedly endorse the use of conda with R projects.

see: http://ihrke.github.io/conda.html


### Where does the environment live on your computer?

Environments are installed by default into the envs directory in your conda directory. This makes it "easy" to know what to version control in your project - your code and the `environment.yml` file.

### Deleting environments

If you want to delete a virtual environment, type:
```    
conda remove --name <environment_name> --all    
```

### Exercise: Try cloning and running a `conda` project

- Clone or download this `conda`-managed Python project: https://github.ubc.ca/timberst/conda_example
- Discuss with your neighbour the pros and cons of using `conda` with Python projects.

### conda wrap-up

- Lighter weight than Docker for managing environments (does not isolate things to the same extent however)
- Works swimmingly with Python and Python packages
- There are challenges to using this with R packages not already on conda/Ananconda (so I recommend cautious dependence in this case)


## Class discussion

- Why is reproducibility is important for data analysis?

- How do I choose the level of investment of reproducibility for a given project?

### Course learning objectives:


By the end of the course, students are expected to be able to:

- Map a data analysis question to appropriate analysis
- Write R, Python and shell scripts for non-interactive data analysis.
- Run literate coding documents (Jupyter notebooks and R Markdown - documents) non-interactively.
- Use a Git/GitHub forking-pull request collaboration approach to collaboratively work on a data analysis project.
- Automate data science workflows (using e.g., Make).
- Manage project software and environment dependencies (using e.g., Docker)
